INTRODUCTION

** Author: Emmanuella Oteng Frimpong

** Assignment 2

#Description

In sports prediction, large numbers of factors including the historical performance of the teams, results of matches, and data on players, have to be accounted for to help different stakeholders understand the odds of winning or losing.

The tasks that are supoosed to be performed are;

Demonstrate the data preparation & feature extraction process
Create feature subsets that show maximum correlation with the dependent variable.
Create and train a suitable machine learning model with cross-validation that can predict a player's rating.
Measure the model's performance and fine-tune it as a process of optimization.
Use the data from another season(players_22) which was not used during the training to test how good is the model.
Deploy the model on a simple web page using either (Heroku, Streamlite, or Flask) and upload a video that shows how the model performs on the web page/site.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Imports and Loading

This section of the notebook will be used for installing and loading datasets and libraries.


In [73]:
!pip install pandas numpy matplotlip seaborn xgboost scikit-learn


In [71]:
!python --version

Python 3.10.12


In [72]:
np.__version__

'1.25.2'

In [74]:
## Importing necessary llibraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import pickle

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, VotingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

After importing the needed libraries, now we have to load datasets.

In [102]:
##loading the datasets.
male_legacy_df = pd.read_csv("/content/drive/MyDrive/INTRO TO AI/male_players (legacy).csv") ## for training the model
player22_df = pd.read_csv("/content/drive/MyDrive/INTRO TO AI/players_22-1.csv") ## for testing the model

<ipython-input-102-08251e6076a0>:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  male_legacy_df = pd.read_csv("/content/drive/MyDrive/INTRO TO AI/male_players (legacy).csv") ## for training the model
<ipython-input-102-08251e6076a0>:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  player22_df = pd.read_csv("/content/drive/MyDrive/INTRO TO AI/players_22-1.csv") ## for testing the model


# Data Preprocessing


## EDA, Imputation and Encoding.

In this section, since we know we have already loaded our data therefore we would be performing an exploratory data analysis, identifying features that are important to us, doing imputation and performing encoding on all the necessary columns.

This step is necessary for the transformation of our data since not all columns, rows are needed for the analysis.

In [103]:
#view first few rows and the nature of the data
male_legacy_df.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [104]:
player22_df.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [105]:
#try to understand the nature oof the data.
male_legacy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [106]:
player22_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 16.1+ MB


From the infomation decribed above, we can see that there are a lot of columns to consider for the analysis. So we are going to need to drop some, before finding the needed features we can work with, let's get the number of missing values for each of our dataframes

In [107]:
#Checking for missing data
print("Checking sum of missing value for Players 21(Train Data:)")
male_legacy_df.isnull().sum()

Checking sum of missing value for Players 21(Train Data:)


player_id           0
player_url          0
fifa_version        0
fifa_update         0
fifa_update_date    0
                   ..
cb                  0
rcb                 0
rb                  0
gk                  0
player_face_url     0
Length: 110, dtype: int64

In [108]:
print("Checking sum of missing value for Players 22(Test Data:)")
player22_df.isnull().sum()

Checking sum of missing value for Players 22(Test Data:)


sofifa_id               0
player_url              0
short_name              0
long_name               0
player_positions        0
                    ...  
player_face_url         0
club_logo_url          61
club_flag_url          61
nation_logo_url     18480
nation_flag_url         0
Length: 110, dtype: int64

## Dropping missing values

Now that we know the information and the columns with lots of missing values, we are going to drop the ones that have 30% or more of the data missing.

In [109]:
total_rows_male = male_legacy_df.shape[0] #shape for train data
total_rows_player22 = player22_df.shape[0] #shape for test data

Calculate the 30% threshhold for the two sets

In [110]:
threshold_male = int(0.3 * total_rows_male)
threshold_player22 = int(0.3 * total_rows_player22)

print("The threshold for Males Legacy is", threshold_male)

The threshold for Males Legacy is 48474


In [111]:
print("The theshold for Players 22 is", threshold_player22)

The theshold for Players 22 is 5771


Get a list of all columns with a sum of missing values greater than the threshold:

In [112]:
columns_to_drop = []
for column in male_legacy_df.columns:
    if male_legacy_df[column].isna().sum() > threshold_male:
        columns_to_drop.append(column)

In [113]:
columns_to_drop_22 = []
for column in player22_df.columns:
    if  player22_df[column].isna().sum() > threshold_player22:
        columns_to_drop_22.append(column)

Drop the columns:

In [114]:
male_legacy_df = male_legacy_df.drop(columns=columns_to_drop, axis=0)
player22_df = player22_df.drop(columns=columns_to_drop_22 , axis=0)

So now leets check the info again to see if the columns dropped.

In [115]:
#understand nature of data
male_legacy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 102 entries, player_id to player_face_url
dtypes: float64(14), int64(45), object(43)
memory usage: 125.7+ MB


In [116]:
player22_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 102 entries, sofifa_id to nation_flag_url
dtypes: float64(13), int64(44), object(45)
memory usage: 15.0+ MB


After reviewing kaggle, reading the data description and looking at things using the data explorer, I came to understand some comuns don't contribute to the overall rating of a player, so we are going to drop those columns too.

Here is a link to exploer the columns in the data: [Data Explorer on Kaggle for Players 22](https://www.kaggle.com/datasets/stefanoleone992/fifa-22-complete-player-dataset/?select=players_22.csv)

and [Data Explorer on Kaggle for Males Legacy](https://www.kaggle.com/datasets/stefanoleone992/fifa-23-complete-player-dataset?select=male_players+%28legacy%29.csv)

For males legacy csv, lets drop those columns.

In [117]:
drop_columns = ['club_contract_valid_until_year','club_name','club_position','club_joined_date','international_reputation','nationality_id','club_team_id','player_id','player_url','fifa_version','long_name','dob','body_type','real_face','player_face_url', 'fifa_update', 'fifa_update_date']

male_legacy_df = male_legacy_df.drop(drop_columns, axis=1)

In [118]:
drop_columns = ['club_contract_valid_until','club_name','club_position','international_reputation','nationality_id','club_team_id','sofifa_id','player_url','long_name','dob','body_type','real_face','player_face_url','club_logo_url','club_flag_url','nation_flag_url']

player22_df = player22_df.drop(drop_columns, axis=1)

After a further review, some columns were identified that could be dropped with this justfication. If we look at the `ls` column it is described as the `player attribute playing as LW`.

Such columns are only useful if we wanted to predict a player's effectiveness in playing such a position, so we drop such columns with that description.


Players are normally played in a specific posiion at their clubs which contibutes more to their overall rating, thus columns like `players_positions` which is the `player preferred positions`


Other columns reviewd that can be dropped are;
*   `short_name`
*   `club_joined`
*   `nationality_name`

In [119]:
#drop new identified columnas
drop_r_cols = ['short_name', 'player_positions', 'league_name', 'nationality_name', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

male_legacy_df  = male_legacy_df .drop(drop_r_cols, axis=1)
player22_df = player22_df.drop(drop_r_cols, axis=1)

## Imputation x Encoding

Having dropped columns with many missing values, we can noe begin our imutation. Imputation is where we will fill missing data with certain values.

In [120]:
## Filling missing numeric data with the mean value
num_imputer = SimpleImputer(strategy='mean')

## Filling missing categorical data with the most frequent value
cat_imputer = SimpleImputer(strategy='most_frequent')

In [121]:
# Selecting numerical and categorical features

num_features_males = male_legacy_df.select_dtypes(include=[np.number]).columns.tolist()
cat_features_males = male_legacy_df.select_dtypes(include=[object]).columns.tolist()

In [122]:
num_features_22 = player22_df.select_dtypes(include=[np.number]).columns.tolist()
cat_features_22 = player22_df.select_dtypes(include=[object]).columns.tolist()

In [123]:
cat_features_males, cat_features_22

(['preferred_foot', 'work_rate'],
 ['club_joined', 'preferred_foot', 'work_rate'])

From Kaggle the column `overall` is described as the *player current overall attribute* which transalte to the **the player rating** i.e the crux of this whole project. Thus we remove `overall` since it is our target variable.

In [124]:
# Removing the target variable from the features
num_features_males.remove('overall')  # 'overall' is the target variable
num_features_22.remove('overall')  # 'overall' is the target variable

Now to the Imputation;


In [125]:
#males legacy imputation
male_legacy_df[num_features_males] = num_imputer.fit_transform(male_legacy_df[num_features_males])

#categorical imputation
male_legacy_df[cat_features_males] = cat_imputer.fit_transform(male_legacy_df[cat_features_males])

In [126]:
#player 22 imputation
player22_df[num_features_22] = num_imputer.fit_transform(player22_df[num_features_22])

#categorical imputation
player22_df[cat_features_22] = cat_imputer.fit_transform(player22_df[cat_features_22])

In [127]:
male_legacy_df.shape

(161583, 54)

In [128]:
player22_df.shape

(19239, 55)

Next task is to perform is the encoding. We do this for only categorical columns. We first explored encoding use OneHot Encoding technique, but quickly discovered that we run out of memory so quickly pivoted to encoding using pd.get_dummies

In [129]:
# Using `get_dummies` for one-hot encoding and dropping the first category
male_legacy_encoded_df = pd.get_dummies(male_legacy_df, columns=cat_features_males, drop_first=True)
player22_encoded_df = pd.get_dummies(player22_df, columns=cat_features_22, drop_first=True)

male_legacy_encoded_df.head()  # display the first few rows to verify the changes

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,club_jersey_number,...,goalkeeping_reflexes,preferred_foot_Right,work_rate_High/Low,work_rate_High/Medium,work_rate_Low/High,work_rate_Low/Low,work_rate_Low/Medium,work_rate_Medium/High,work_rate_Medium/Low,work_rate_Medium/Medium
0,93,95.0,100500000.0,550000.0,27.0,169.0,67.0,53.0,1.0,10.0,...,8.0,False,False,False,False,False,False,False,True,False
1,92,92.0,79000000.0,375000.0,29.0,185.0,80.0,53.0,1.0,7.0,...,11.0,True,True,False,False,False,False,False,False,False
2,90,90.0,54500000.0,275000.0,30.0,180.0,80.0,19.0,1.0,10.0,...,15.0,False,True,False,False,False,False,False,False,False
3,90,90.0,52500000.0,275000.0,32.0,195.0,95.0,16.0,1.0,10.0,...,12.0,True,False,False,False,False,False,False,True,False
4,90,90.0,63500000.0,300000.0,28.0,193.0,92.0,19.0,1.0,1.0,...,86.0,True,False,False,False,False,False,False,False,True


In [130]:
male_legacy_encoded_df.shape

(161583, 61)

Finally, we can describe our datasets before we start the feature analysis.

In [131]:
male_legacy_encoded_df.describe

<bound method NDFrame.describe of         overall  potential    value_eur  wage_eur   age  height_cm  weight_kg  \
0            93       95.0  100500000.0  550000.0  27.0      169.0       67.0   
1            92       92.0   79000000.0  375000.0  29.0      185.0       80.0   
2            90       90.0   54500000.0  275000.0  30.0      180.0       80.0   
3            90       90.0   52500000.0  275000.0  32.0      195.0       95.0   
4            90       90.0   63500000.0  300000.0  28.0      193.0       92.0   
...         ...        ...          ...       ...   ...        ...        ...   
161578       46       61.0     110000.0     700.0  18.0      180.0       73.0   
161579       46       58.0     110000.0     750.0  19.0      188.0       83.0   
161580       46       58.0     110000.0     500.0  19.0      181.0       73.0   
161581       46       70.0     150000.0     500.0  17.0      175.0       68.0   
161582       46       63.0     110000.0     500.0  17.0      180.0       70.0   

        league_id  league_level  club_jersey_number  ...  \
0            53.0           1.0                10.0  ...   
1            53.0           1.0                 7.0  ...   
2            19.0           1.0                10.0  ...   
3            16.0           1.0                10.0  ...   
4            19.0           1.0                 1.0  ...   
...           ...           ...                 ...  ...   
161578     2012.0           1.0                45.0  ...   
161579     2012.0           1.0                41.0  ...   
161580       65.0           1.0                34.0  ...   
161581       65.0           1.0                28.0  ...   
161582      350.0           1.0                42.0  ...   

        goalkeeping_reflexes  preferred_foot_Right  work_rate_High/Low  \
0                        8.0                 False               False   
1                       11.0                  True                True   
2                       15.0                 False                True   
3                       12.0                  True               False   
4                       86.0                  True               False   
...                      ...                   ...                 ...   
161578                   7.0                  True               False   
161579                  14.0                 False               False   
161580                   9.0                  True               False   
161581                  13.0                  True               False   
161582                  14.0                  True               False   

        work_rate_High/Medium  work_rate_Low/High  work_rate_Low/Low  \
0                       False               False              False   
1                       False               False              False   
2                       False               False              False   
3                       False               False              False   
4                       False               False              False   
...                       ...                 ...                ...   
161578                  False               False              False   
161579                  False               False              False   
161580                   True               False              False   
161581                  False               False              False   
161582                  False               False              False   

        work_rate_Low/Medium  work_rate_Medium/High  work_rate_Medium/Low  \
0                      False                  False                  True   
1                      False                  False                 False   
2                      False                  False                 False   
3                      False                  False                  True   
4                      False                  False                 False   
...                      ...                    ...     

In [132]:
player22_encoded_df.describe

<bound method NDFrame.describe of        overall  potential    value_eur  wage_eur   age  height_cm  weight_kg  \
0           93       93.0   78000000.0  320000.0  34.0      170.0       72.0   
1           92       92.0  119500000.0  270000.0  32.0      185.0       81.0   
2           91       91.0   45000000.0  270000.0  36.0      187.0       83.0   
3           91       91.0  129000000.0  270000.0  29.0      175.0       68.0   
4           91       91.0  125500000.0  350000.0  30.0      181.0       70.0   
...        ...        ...          ...       ...   ...        ...        ...   
19234       47       52.0      70000.0    1000.0  22.0      180.0       64.0   
19235       47       59.0     110000.0     500.0  19.0      175.0       70.0   
19236       47       55.0     100000.0     500.0  21.0      178.0       72.0   
19237       47       60.0     110000.0     500.0  19.0      173.0       66.0   
19238       47       60.0     110000.0     500.0  19.0      167.0       61.0   

       league_level  club_jersey_number  weak_foot  ...  \
0               1.0                30.0        4.0  ...   
1               1.0                 9.0        4.0  ...   
2               1.0                 7.0        4.0  ...   
3               1.0                10.0        5.0  ...   
4               1.0                17.0        5.0  ...   
...             ...                 ...        ...  ...   
19234           1.0                36.0        3.0  ...   
19235           1.0                27.0        3.0  ...   
19236           1.0                31.0        3.0  ...   
19237           1.0                12.0        3.0  ...   
19238           1.0                13.0        3.0  ...   

       club_joined_2021-09-15  preferred_foot_Right  work_rate_High/Low  \
0                       False                 False               False   
1                       False                  True               False   
2                       False                  True                True   
3                       False                  True               False   
4                       False                  True               False   
...                       ...                   ...                 ...   
19234                   False                  True               False   
19235                   False                  True               False   
19236                   False                  True               False   
19237                   False                  True               False   
19238                   False                  True               False   

       work_rate_High/Medium  work_rate_Low/High  work_rate_Low/Low  \
0                      False               False              False   
1                       True               False              False   
2                      False               False              False   
3                       True               False              False   
4                      False               False              False   
...                      ...                 ...                ...   
19234                  False               False              False   
19235                  False               False              False   
19236                  False               False              False   
19237                  False               False              False   
19238                  False               False              False   

       work_rate_Low/Medium  work_rate_Medium/High  work_rate_Medium/Low  \
0                     False                  False                  True   
1                     False                  False                 False   
2                     False                  False                 False   
3                     False                  False                 False   
4                     False                  False                 False   
...                     ...                    ...                   ...   
19234     

#Feature Engineering.


## Feature Extraction
Now we are going to analyze the dataset to understand which features are important for determining a player's overall rating. We are using feature importance *to* identify necessary features.


Here we are fitting a RandomForestRegressor to obtain feature importances.

In [133]:
# the target variable and features; drop non-numeric columns if necessary
X = male_legacy_encoded_df.drop(columns=['overall'])
y = male_legacy_encoded_df['overall']

In [134]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)

In [135]:
# Create a Random Forest Regressor model
model = RandomForestRegressor(random_state=69)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=69)

In [136]:
# Get feature importances
importances = model.feature_importances_

# Sort them in descending order
indices = np.argsort(importances)[::-1]

# Let's print out the feature importance ranking
print("Top 20 Feature ranking:")

for i in range(21):
    print(f"{i + 1}. Feature {X.columns[indices[i]]} ({importances[indices[i]]})")

Top 20 Feature ranking:
1. Feature value_eur (0.8065332959425644)
2. Feature age (0.08062255967101842)
3. Feature potential (0.04557715087721492)
4. Feature movement_reactions (0.02080080413408383)
5. Feature wage_eur (0.014259375005116369)
6. Feature mentality_composure (0.006396275740171184)
7. Feature defending (0.004508892564411371)
8. Feature dribbling (0.0026127347611774937)
9. Feature skill_ball_control (0.0012526488480551143)
10. Feature physic (0.0009574911442534491)
11. Feature attacking_crossing (0.0008622794809781083)
12. Feature power_stamina (0.0008228267629681519)
13. Feature shooting (0.0006882887980096972)
14. Feature mentality_positioning (0.0006503894634027378)
15. Feature goalkeeping_positioning (0.0005986013742359419)
16. Feature power_shot_power (0.0005732124986454873)
17. Feature goalkeeping_diving (0.0005595093003058266)
18. Feature attacking_heading_accuracy (0.0005566025428797454)
19. Feature league_id (0.0005496121814647509)
20. Feature passing (0.00053297694

In [137]:
#Now, let's get the top 10 features
top_features = [X.columns[indices[i]] for i in range(10)]
print("\nTop 10 features with % Contribution:")

for i in range(10):
    print(f"{i + 1}.  {top_features[i]} ({round(importances[indices[i]]*100,2)}%)")


Top 10 features with % Contribution:
1.  value_eur (80.65%)
2.  age (8.06%)
3.  potential (4.56%)
4.  movement_reactions (2.08%)
5.  wage_eur (1.43%)
6.  mentality_composure (0.64%)
7.  defending (0.45%)
8.  dribbling (0.26%)
9.  skill_ball_control (0.13%)
10.  physic (0.1%)


Observing the results of the feature importance process, it can be seen that the top 5 features contribute a percentage importance of *97%*.

Thus my strategy is to use the top 10 features to train so I capture the underlying data patterns even for weak contributing features. Then when testing use the same 5. And, when deployed in the future use the top 5 features for prediction.

Let's see how it Goes. On to Feature subsetting.

In [138]:
top_features = top_features[:5]

print('Features being used for model development are:\n')
top_features

Features being used for model development are:



['value_eur', 'age', 'potential', 'movement_reactions', 'wage_eur']

## Feature Subset

At this stage our goal is to use the top features we have identified during our feature extraction stage to create subsetted data that we will use to train models.

In [139]:
#Now we subset our X feauture set
X_top_f = X[top_features]
X_top_f

#no need to do for y

,value_eur,age,potential,movement_reactions,wage_eur
0,100500000.0,27.0,95.0,94.0,550000.0
1,79000000.0,29.0,92.0,90.0,375000.0
2,54500000.0,30.0,90.0,89.0,275000.0
3,52500000.0,32.0,90.0,85.0,275000.0
4,63500000.0,28.0,90.0,89.0,300000.0
...,...,...,...,...,...
161578,110000.0,18.0,61.0,39.0,700.0
161579,110000.0,19.0,58.0,42.0,750.0
161580,110000.0,19.0,58.0,50.0,500.0
161581,150000.0,17.0,70.0,45.0,500.0


Now lets scale our features which is our independent variables

In [140]:
# Initialize the scaler
scaler = StandardScaler()

# Scale the features
X_scaled = scaler.fit_transform(X_top_f)

# The features are now scaled and ready for training the model.
X_scaled_df = pd.DataFrame(X_scaled, columns=X_top_f.columns)

X_scaled_df.head()

,value_eur,age,potential,movement_reactions,wage_eur
0,16.451449,0.401872,3.875315,3.535775,24.706815
1,12.848571,0.830120,3.396013,3.099571,16.687273
2,8.742966,1.044244,3.076478,2.990520,12.104678
3,8.407815,1.472491,3.076478,2.554317,12.104678
4,10.251147,0.615996,3.076478,2.990520,13.250326


In [141]:
#Saving scaler to use in deployment

with open('/content/drive/MyDrive/INTRO TO AI/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


# Training Models

We are now reading to train some models, here we are going to train 3 modes;
1. XGBoost
2. Gradient Boost
3. Random Forest


Lets split data for training

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=84)

Now we define a function for our different training models.


In [ ]:
def train_model(model, param_grid, X, y):
    '''
        Trains a model using grid search with cross-validation and returns the best model.
        Parameters:
            model: scikit-learn model
            param_grid: dictionary with parameters to try
            X: features(independent variables)
            y: target(dependent variable)
    '''
    cv = KFold(n_splits=7 , random_state=69, shuffle=True)

    # Grid search with cross-validation
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
    grid_search.fit(X, y)

    # Results of the grid search
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best score (MAE): {-grid_search.best_score_}")  # We negate the score because grid search maximizes performance (so it negates the scores)

    return grid_search.best_estimator_  # Returns the best model

## Model 1: XGBoost


In [ ]:
print("\nTraining XGBoost...")
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_params = {
    'n_estimators': [10, 50, 100],
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5, 15],
    'colsample_bytree': [0.5,1]
}
best_xgb = train_model(xgb_model, xgb_params, X_train, y_train)


Training XGBoost...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 100}
Best score (MAE): 0.6224623308863931


##Model 2: Gradient Boosting Regressor

In [ ]:
print("\nTraining Gradient Boosting...")
gbr_model = GradientBoostingRegressor(random_state=63)
gbr_params = {
    'n_estimators': [10, 50,  100,],
    'learning_rate': [0.1, 0.01],
    'max_depth': [9, 15]
}
best_gbr = train_model(gbr_model, gbr_params, X_train, y_train)


Training Gradient Boosting...
Best parameters: {'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 50}
Best score (MAE): 0.6389476703651458


## Model 3: Random Forest


In [ ]:
print("\nTraining Random Forest...")
rf_model = RandomForestRegressor(random_state=39)
rf_params = {
    'n_estimators': [10, 50, 100],
    'max_depth': [12, 15],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
best_rf = train_model(rf_model, rf_params, X_train, y_train)


Training Random Forest...
Best parameters: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best score (MAE): 0.620120615885473


## Model 4: Ensembled Model
Form discussion in class I have come to understand that, an ensembled model can improve a model's predicitve perfromance. Here I will combine the best versions of my 3 models into a single ensemble model.

In [ ]:
# Create an ensemble model
ensemble = VotingRegressor(
    estimators=[
        ('xgb', best_xgb),
        ('gbr', best_gbr),
        ('rf', best_rf)
    ]
)

In [ ]:
# Fit model on the training data
print("\nTraining Ensemble Model...")
ensemble.fit(X_train, y_train)

# Predict and evaluate on the training set
train_pred = ensemble.predict(X_train)
train_mae = mean_absolute_error(y_train, train_pred)
print(f"Ensemble model MAE on training set: {train_mae}")


Training Ensemble Model...
Ensemble model MAE on training set: 0.30503444733299373


### Loading Saved Models

In [ ]:
%cd '/content/drive/MyDrive/Intro to ai'

[Errno 2] No such file or directory: '/content/drive/MyDrive/Intro to ai'
/content


In [ ]:
with open('best_xgb_model.pkl', 'wb') as file:
    pickle.dump(best_xgb, file)

with open('best_gbr_model.pkl', 'wb') as file:
    pickle.dump(best_gbr, file)

with open('best_rf_model.pkl', 'wb') as file:
    pickle.dump(best_rf, file)

with open('ensemble_model.pkl', 'wb') as file:
    pickle.dump(ensemble, file)

In [ ]:
with open('ensemble_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

predictions = loaded_model.predict(X_test)

en_mae = mean_absolute_error(y_test, predictions)

print(f"Ensemble model MAE on test set: {en_mae}")

Ensemble model MAE on test set: 0.6036230979910489


### Testing

In [ ]:
print("\nEvaluating XGBoost...")

#predict on test set
pred_xgb = best_xgb.predict(X_test)
xgb_mae = mean_absolute_error(y_test, pred_xgb)

print(f"XGBoost model MAE on test set: {xgb_mae:.2f}")


Evaluating XGBoost...
XGBoost model MAE on test set: 0.62


In [ ]:
print("\nEvaluating Gradient Boost...")

#predict on test set
pred_gbr = best_gbr.predict(X_test)
gbr_mae = mean_absolute_error(y_test, pred_gbr)

print(f"Gradient Boost Regressor model MAE on test set: {gbr_mae}:.2f")


Evaluating Gradient Boost...
Gradient Boost Regressor model MAE on test set: 0.6283813246353903:.2f


In [ ]:
print("\nEvaluating Ensemble...")

#predict on test set
pred_en = ensemble.predict(X_test)
en_mae = mean_absolute_error(y_test, pred_en)

print(f"Ensemble model MAE on test set: {en_mae:.2f}")


Evaluating Ensemble...
Ensemble model MAE on test set: 0.60


In [ ]:
!pip freeze > requirements.txt